In [ ]:
# Import all necessary libraries

import keras 
from keras.saving import serialize_keras_object
from tensorflow.keras import utils
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten
import json

In [ ]:
# Declare our start time and end time

start = datetime(2020, 1, 1)
end = datetime(2022, 1, 1)

In [ ]:
# Access the quantbook API and define the asset a variable, create the data frame based on the asset

qb = QuantBook()
symbol = qb.AddCrypto("BTCUSD", Resolution.Daily).Symbol
history = qb.History(symbol, start, end).loc[symbol]
history.head() # Previews first 5 rows 

In [ ]:
# Alter the data frame and take only specific columns, also return the percentage change

daily_pct_change = history[["open", "high", "low", "close", "volume"]].pct_change().dropna()
df = daily_pct_change
df.head() 

In [ ]:
# Removes all values of infinitiy and replaces with the highest finite value
indexes = df[((df.volume == float("inf")))].index
for i in indexes:
    df.at[i, "volume"] = max(df.volume.drop(indexes)) 

In [ ]:
# Take last thirty days and make a list, this will be used for each decision
# Go to the day we are examining and return if it was a losing or winning date 

n_steps = 30 # Step size (data per step)
features = [] # inputs
labels = [] # outputs 
for i in range(len(df)-n_steps):
    input_data = df.iloc[i:i+n_steps].values
    features.append(input_data)
    if df['close'].iloc[i+n_steps] >= 0:
        # UP
        label = 1
    else: 
        # DOWN
        label = 0
    labels.append(label) 

In [ ]:
# Changing the type of the lists from primative into numpy arrays which is a requirement for keras 

features = np.array(features) # Convert to desired input type
labels = np.array(labels) 

In [ ]:
# Define our training and testing data sets, features is the outputs and labels are our inputs 

train_length = int(len(features) * 0.7) # split the data for training and testing
x_train = features[:train_length]
x_test = features[train_length:]
y_train = labels[:train_length]
y_test = labels[train_length:]

In [ ]:
# Number of up vs down days in training data should be relatively balanced

sum(y_train)/len(y_train)

In [ ]:
# Use second part of data for training instead

test_length = int(len(features) * 0.3)
x_train = features[test_length:]
x_test = features[:test_length]
y_train = labels[test_length:]
y_test = labels[:test_length]

In [ ]:
# Number of up vs down days in training data should be relatively balanced

sum(y_train)/len(y_train)

In [ ]:
# features, shape, relu gets rid of negative

model = Sequential([Dense(30, input_shape=x_train[0].shape, activation='relu'),
    Dense(20, activation='relu'),
    Flatten(),
    Dense(1, activation='sigmoid')])

In [ ]:
model.compile(loss='binary_crossentropy',
                    optimizer='adam',
                    metrics=['accuracy', 'mse'])

In [ ]:
# Epochs is how many times it loops through the data

model.fit(x_train, y_train, epochs=5)

In [ ]:
# Now test the model

y_hat = model.predict(x_test)

In [ ]:
results = pd.DataFrame({'y': y_test.flatten(), 'y_hat': y_hat.flatten()})

In [ ]:
results.plot(title='Model Performance: predicted vs actual %change in closing price', figsize=(17, 7))

In [ ]:
pred_train = model.predict(x_train)
scores = model.evaluate(x_train, y_train, verbose=0)
print('Accuracy on training data: {}% \n Error on training data: {}'.format(scores[1], 1 - scores[1]))

pred_test= model.predict(x_test)
scores2 = model.evaluate(x_test, y_test, verbose=0)
print('Accuracy on test data: {}% \n Error on test data: {}'.format(scores2[1], 1 - scores2[1]))